In [1]:
import ee
ee.Authenticate()

Enter verification code:  4/1AeanS0ZAwT7W5pxEtOBzEFfTsKkCgZTWNk3EuqWJP-blymBZWbhhlCb6_jQ



Successfully saved authorization token.


In [2]:
ee.Initialize()

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


In [19]:
import ee
import folium

# Initialize the Earth Engine module
ee.Initialize()

# Define the South Florida boundary using the provided coordinates
south_florida = ee.Geometry.Rectangle([
    -81.562709,  # lon_min
    25.022916,   # lat_min
    -79.899069,  # lon_max
    26.092407    # lat_max
])

# Load the MODIS Land Surface Temperature dataset (MOD11A1)
lst_dataset = ee.ImageCollection('MODIS/006/MOD11A1') \
    .filterBounds(south_florida) \
    .filterDate('2023-01-01', '2023-12-31') \
    .select('LST_Day_1km') \
    .mean()  # Take the average LST for the year

# Convert the LST from Kelvin to Celsius by subtracting 273.15
lst_celsius = lst_dataset.select('LST_Day_1km').subtract(273.15).rename('LST_Celsius')

# Define visualization parameters for Surface Temperature (LST)
lst_vis_params = {
    'min': 20,  # Minimum temperature (°C)
    'max': 40,  # Maximum temperature (°C)
    'palette': ['blue', 'green', 'yellow', 'red']  # Color ramp from cool to hot
}

# Function to add an Earth Engine layer to a folium map
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add Earth Engine layer function to folium
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map centered on South Florida
map_center = [(25.022916 + 26.092407) / 2, (-81.562709 + -79.899069) / 2]  # Approximate center of the rectangle
mymap = folium.Map(location=map_center, zoom_start=8)

# Add the Surface Temperature (LST) layer to the map
mymap.add_ee_layer(lst_celsius, lst_vis_params, 'Surface Temperature (LST)')

# Add the South Florida boundary to the map
folium.GeoJson(south_florida.getInfo(), name='South Florida Boundary').add_to(mymap)

# Add layer control and display the map
folium.LayerControl().add_to(mymap)
mymap


EEException: Image.select: Band pattern 'LST_Day_1km' was applied to an Image with no bands. See https://developers.google.com/earth-engine/guides/debugging#no-bands

In [20]:
import ee
import folium

# Initialize the Earth Engine module
ee.Initialize()

# Define the South Florida boundary using the provided coordinates
south_florida = ee.Geometry.Rectangle([
    -81.562709,  # lon_min
    25.022916,   # lat_min
    -79.899069,  # lon_max
    26.092407    # lat_max
])

# Load the MODIS Land Surface Temperature dataset (MOD11A1)
lst_dataset = ee.ImageCollection('MODIS/006/MOD11A1') \
    .filterBounds(south_florida) \
    .filterDate('2023-01-01', '2023-12-31')

# Check if the collection is not empty and get an image from the collection
lst_image = lst_dataset.mean().select('LST_Day_1km')  # Taking the mean and selecting the LST band

# Check if the selected band exists
band_names = lst_image.bandNames().getInfo()
if 'LST_Day_1km' not in band_names:
    print("Error: 'LST_Day_1km' band is not available.")
else:
    # Convert the LST from Kelvin to Celsius by subtracting 273.15
    lst_celsius = lst_image.subtract(273.15).rename('LST_Celsius')

    # Define visualization parameters for Surface Temperature (LST)
    lst_vis_params = {
        'min': 20,  # Minimum temperature (°C)
        'max': 40,  # Maximum temperature (°C)
        'palette': ['blue', 'green', 'yellow', 'red']  # Color ramp from cool to hot
    }

    # Function to add an Earth Engine layer to a folium map
    def add_ee_layer(self, ee_image_object, vis_params, name):
        map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
        folium.raster_layers.TileLayer(
            tiles=map_id_dict['tile_fetcher'].url_format,
            attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
            name=name,
            overlay=True,
            control=True
        ).add_to(self)

    # Add Earth Engine layer function to folium
    folium.Map.add_ee_layer = add_ee_layer

    # Create a folium map centered on South Florida
    map_center = [(25.022916 + 26.092407) / 2, (-81.562709 + -79.899069) / 2]  # Approximate center of the rectangle
    mymap = folium.Map(location=map_center, zoom_start=8)

    # Add the Surface Temperature (LST) layer to the map
    mymap.add_ee_layer(lst_celsius, lst_vis_params, 'Surface Temperature (LST)')

    # Add the South Florida boundary to the map
    folium.GeoJson(south_florida.getInfo(), name='South Florida Boundary').add_to(mymap)

    # Add layer control and display the map
    folium.LayerControl().add_to(mymap)
    mymap


EEException: Image.select: Band pattern 'LST_Day_1km' was applied to an Image with no bands. See https://developers.google.com/earth-engine/guides/debugging#no-bands